<a href="https://colab.research.google.com/github/Zhejing-Chin/fyp/blob/main/IoU_Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import pandas as pd

In [ ]:
def calc_iou(boxA, boxB):
  # boxA - gt, boxB - pred
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
 
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
 
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
 
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
 
	# return the intersection over union value
	return iou

In [ ]:
# keep only 27 classes
classes = ['Bakery', 'Biscuits', 'Bonbons', 'Canned', 'Cereals', 'Cheese', 'Chips', 'Choco', 'Chocolate', 'Coffee', 'Creme', 'DriedFruitsAndNuts', 'IceTea', 'Juices', 'Milk', 'Oil-Vinegar', 'Pasta', 'Rice', 'Sauces', 'Snacks', 'SoftDrinks', 'Soups', 'Spices', 'Spreads', 'Tea', 'Water', 'Yoghurt']

path = '/content/drive/MyDrive/FYP/Detection/'
prediction_csv = 'res_tesco_0.5_2021-02-03 03:25:04.625617.csv'
filtered_iou = []

prediction = pd.read_csv(path + prediction_csv, 
                         header=0)
groundtruth = pd.read_csv("/content/drive/MyDrive/FYP/New Data/Tesco/NewDataAnnotations.csv",
                          header=None, 
                          names=['image_path', 'x1', 'y1', 'x2', 'y2', 'class_', 'width', 'height'])
threshold = 0.3

#read each predicted box(pb) in prediction csv
for pred in prediction.itertuples():
  for gt in groundtruth.itertuples():
    if gt.class_ not in classes:
      continue
    #compare each pb with every groundtruth box(gtb) from the same image
    if gt.image_path == pred.image_id:

      #Calculate the IoU
      iou = calc_iou(list([gt.x1,gt.y1,gt.x2,gt.y2]), list([pred.x1,pred.y1,pred.x2,pred.y2])) #take x1, y1, x2, y2
      
      #keep only the pb which > IoU threshold
      if iou < threshold:
        continue
      else:
        prediction.at[pred.Index, 'iou'] = iou
        filtered_iou.append(
            list([pred.image_id, pred.x1,pred.y1,pred.x2,pred.y2, gt.class_, iou])
            )

# confusion matrix

In [ ]:
# filtered_iou
df=pd.DataFrame(filtered_iou,columns=['image_id', 'x1', 'y1', 'x2', 'y2', 'class', 'iou'])
df.to_csv('/content/drive/MyDrive/FYP/detection_results_filtered_iou/' + prediction_csv, 
          header = False,
          index = False)
len(df['class'].unique())


27

# Extract objects from detection result


In [ ]:
import os

In [ ]:
# create a directory naming with detection_results_filtered_iou csv file

def create_folder(path):
    """
    Checks if the path exists, if not creates it.
    :param path: A valid path that might not exist
    :return: An indication if the folder was created
    """
    folder_missing = not os.path.exists(path)

    if folder_missing:
        # Using makedirs since the path hierarchy might not fully exist.
        try:
            os.makedirs(path)
        except OSError as e:
            if (e.errno, e.strerror) == FILE_EXISTS_ERROR:
                print(e)
            else:
                raise

        print('Created folder {0}'.format(path))

    return folder_missing


def showSubplotImage(img, save_path="", titlestr=""):
    # plt.clf()
    print(titlestr)
    plt.imshow(img, interpolation='bilinear')
    plt.xticks([]), plt.yticks([])  
    try:
      os.makedirs(save_path)
    except OSError as e:
      if e.errno != errno.EEXIST:
          raise
    plt.savefig(save_path + '/' + titlestr, bbox_inches='tight', transparent=True, pad_inches=0)
    
    plt.title(titlestr)
    plt.close()
    # plt.show()

In [ ]:
# CSV file
import cv2 
import matplotlib.pyplot as plt
import errno

detection_path = '/content/drive/MyDrive/FYP/detection_results_filtered_iou/'
csv = prediction_csv

save_path = detection_path + csv.split('.csv')[0]
create_folder(save_path)
annotation = pd.read_csv(detection_path + csv,
                         usecols = [i for i in range(6)],
                         header = None,
                         names=["image_name", "x1", "y1", "x2", "y2", "class_"])

for row in annotation.itertuples():
  # subpath = row.image_name.split("_")[0]
  image_path = '/content/drive/MyDrive/FYP/New Data/Tesco/'
  img = cv2.imread(image_path + row.image_name)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  try:
    showSubplotImage(img[row.y1:row.y2, row.x1:row.x2], save_path+'/'+row.class_, str(row.Index)+'_'+row.image_name)
  except ValueError:  
    pass
  # break

Created folder /content/drive/MyDrive/FYP/detection_results_filtered_iou/res_tesco_0.5_2021-02-03 03:25:04.625617
0_20201222_184846.jpg
1_20201222_184846.jpg
2_20201222_184846.jpg
3_20201222_184846.jpg
4_20201222_184846.jpg
5_20201222_184846.jpg
6_20201222_184846.jpg
7_144928402_163512872006659_6167652525977630073_n.jpg
8_144928402_163512872006659_6167652525977630073_n.jpg
9_144928402_163512872006659_6167652525977630073_n.jpg
10_144928402_163512872006659_6167652525977630073_n.jpg
11_144928402_163512872006659_6167652525977630073_n.jpg
12_144928402_163512872006659_6167652525977630073_n.jpg
13_144928402_163512872006659_6167652525977630073_n.jpg
14_144928402_163512872006659_6167652525977630073_n.jpg
15_144928402_163512872006659_6167652525977630073_n.jpg
16_144928402_163512872006659_6167652525977630073_n.jpg
17_144928402_163512872006659_6167652525977630073_n.jpg
18_144928402_163512872006659_6167652525977630073_n.jpg
19_144928402_163512872006659_6167652525977630073_n.jpg
20_144928402_1635128